In [7]:
# Cell 0: Set Environment Variables

import os

# Set FINN_BUILD_DIR to the build output directory
os.environ["FINN_BUILD_DIR"] = os.path.abspath("../build")

print(f"✓ FINN_BUILD_DIR set to: {os.environ['FINN_BUILD_DIR']}")

✓ FINN_BUILD_DIR set to: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build


In [1]:
# Cell 1: Imports and Setup

import os
import sys
import shutil
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper

# FINN is installed via pip, no need to add to path
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
from finn.builder.build_dataflow_config import DataflowBuildConfig

print("✓ Imports successful")
print(f"  Python: {sys.version}")
print(f"  Working directory: {os.getcwd()}")

# Test FINN
try:
    from finn.util.basic import get_finn_root
    print(f"✓ FINN root: {get_finn_root()}")
except:
    print("✓ FINN imported successfully")

W1203 02:58:21.663359 12506 site-packages/torch/utils/cpp_extension.py:118] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


✓ Imports successful
  Python: 3.9.25 (main, Nov  3 2025, 22:33:05) 
[GCC 11.2.0]
  Working directory: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/notebooks
✓ FINN imported successfully


In [2]:
# Cell 2: Verify Model Exists

# Path to your streamlined QONNX model
model_path = "../../finn_build/ellipse_qonnx_streamlined.qonnx"

# Check if model exists
if not os.path.exists(model_path):
    raise FileNotFoundError(
        f"Model not found: {model_path}\n"
        "Please run 2-finn.ipynb first to generate the streamlined model."
    )

# Load and inspect the model
model = ModelWrapper(model_path)
print(f"✓ Model loaded: {model_path}")
print(f"  Input: {model.graph.input[0].name} {[d.dim_value for d in model.graph.input[0].type.tensor_type.shape.dim]}")
print(f"  Output: {model.graph.output[0].name} {[d.dim_value for d in model.graph.output[0].type.tensor_type.shape.dim]}")
print(f"  Total nodes: {len(model.graph.node)}")

# Count node types
from collections import Counter
node_types = Counter([n.op_type for n in model.graph.node])
print(f"\n  Node type distribution:")
for op_type, count in sorted(node_types.items(), key=lambda x: -x[1])[:10]:
    print(f"    {op_type}: {count}")

✓ Model loaded: ../../finn_build/ellipse_qonnx_streamlined.qonnx
  Input: global_in [0, 1, 20, 20]
  Output: global_out [1, 5]
  Total nodes: 63

  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3


In [3]:
# Cell 3: Configure Build Settings

# Output directory for build artifacts
output_dir = "../build"

# Clean output directory if it exists
if os.path.exists(output_dir):
    print(f"  Cleaning existing build directory: {output_dir}")
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

print(f"✓ Build configuration:")
print(f"  Model: {model_path}")
print(f"  Output: {output_dir}")
print(f"  Target board: Kria KV260")

✓ Build configuration:
  Model: ../../finn_build/ellipse_qonnx_streamlined.qonnx
  Output: ../build
  Target board: Kria KV260


In [5]:
# Cell 4: Create Dataflow Build Configuration

# Create a build config for the KV260 board
cfg = DataflowBuildConfig(
    output_dir=output_dir,
    
    # Target board
    board="Kria-KV260-SOM",  # Full board name for KV260
    
    # Clock frequency (200 MHz = 5.0 ns period)
    synth_clk_period_ns=5.0,
    
    # Build steps - you can customize which steps to run
    steps=build_cfg.default_build_dataflow_steps,
    
    # Resource optimization strategy
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.OOC_SYNTH,
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    
    # Performance settings
    auto_fifo_depths=True,
    auto_fifo_strategy=build_cfg.AutoFIFOSizingMethod.LARGEFIFO_RTLSIM,
    
    # Parallelization settings (adjust based on your model)
    folding_config_file=None,  # Auto-generate folding config
    
    # Enable verbose output
    verbose=True,
    
    # Save intermediate models
    save_intermediate_models=True,
)

print("✓ Build configuration created")
print(f"\n  Build steps ({len(cfg.steps)}):")
for i, step in enumerate(cfg.steps, 1):
    print(f"    {i}. {step}")

print(f"\n  Output types:")
for output_type in cfg.generate_outputs:
    print(f"    - {output_type.name}")

✓ Build configuration created

  Build steps (19):
    1. step_qonnx_to_finn
    2. step_tidy_up
    3. step_streamline
    4. step_convert_to_hw
    5. step_create_dataflow_partition
    6. step_specialize_layers
    7. step_target_fps_parallelization
    8. step_apply_folding_config
    9. step_minimize_bit_width
    10. step_generate_estimate_reports
    11. step_hw_codegen
    12. step_hw_ipgen
    13. step_set_fifo_depths
    14. step_create_stitched_ip
    15. step_measure_rtlsim_performance
    16. step_out_of_context_synthesis
    17. step_synthesize_bitfile
    18. step_make_pynq_driver
    19. step_deployment_package

  Output types:
    - ESTIMATE_REPORTS
    - STITCHED_IP
    - RTLSIM_PERFORMANCE
    - OOC_SYNTH
    - BITFILE
    - DEPLOYMENT_PACKAGE
